In [1]:
"""
TO PROCESS THE NUIM COMPLETE DATASET FOR SWIN TRANSFORMER TRAINING
AUTHOR: PRAMIT DUTTA
EMAIL: PRAMIT.DUTTA.2021@MUMAIL.IE

THE ORIGINAL CLASSES ARE MAPPED TO THE NEW DEFINITION PROVIDED IN NAME_MAPPING DICTIONARY TO 11 CLASSES INCLUDING BACKGROUND

"""
from pathlib import Path
import numpy as np
# import torch
from PIL import Image
from nuimages import NuImages
from nuimages.utils.utils import mask_decode
import os
import os.path as osp 
from matplotlib.pyplot import imshow
# import mmcv
from nuimages.utils.utils import mask_decode, name_to_index_mapping
import shutil

In [2]:
#NUIM DATAROOT
DATAROOT = '/root/Internship-Valeo/Project/data/nuimages'
VERSIONS = ['v1.0-train','v1.0-test','v1.0-val']
# VERSIONS = ['v1.0-trainval']

#PROCESSED DATA_ROOT
data_root='/root/Internship-Valeo/Project/data/nuimages/seg_data/'
ann_dir = data_root+'ann_dir'
img_dir = data_root+'img_dir'
split_dir = data_root+'splits'

In [3]:
if os.path.exists(ann_dir) and os.path.isdir(ann_dir):
    shutil.rmtree(ann_dir)
os.mkdir(ann_dir)

In [4]:
# NAME_MAPPING = {
#     #1
#     'noise':'background',
#     'vehicle.ego':'background',
#     #2
#     'flat.driveable_surface':'lane',
#     #3
#     'movable_object.barrier': 'barrier',
#     'movable_object.debris' : 'barrier',
#     'movable_object.trafficcone': 'barrier',
#     'movable_object.pushable_pullable' : 'barrier',
#     #4
#     'vehicle.bicycle': 'bicycle',
#     #5
#     'vehicle.bus.bendy': 'bus',
#     'vehicle.bus.rigid': 'bus',
#     'vehicle.emergency.ambulance':'bus',
#     'vehicle.emergency.police':'bus',
#     #6    
#     'vehicle.car': 'car',
#     #7
#     'vehicle.construction': 'construction_vehicle',
#     #8
#     'vehicle.motorcycle': 'motorcycle',
#     #9
#     'vehicle.trailer': 'trailer',
#     #10
#     'vehicle.truck': 'truck',
#     #11
#     'human.pedestrian.adult': 'pedestrian',
#     'human.pedestrian.child': 'pedestrian',
#     'human.pedestrian.construction_worker': 'pedestrian',
#     'human.pedestrian.police_officer': 'pedestrian',
#     'human.pedestrian.personal_mobility':'pedestrian',
#     'human.pedestrian.stroller':'pedestrian',
#     'human.pedestrian.wheelchair':'pedestrian',
#     #12
#     'static.manmade':'static',
#     'static.other':'static',
#     'static.vegetation':'static',
#     'static_object.bicycle_rack':'static',
#     #13
#     'flat.other':'other_surface',
#     'flat.sidewalk':'other_surface',
#     'flat.terrain':'other_surface',
#     #14
#     'animal':'animal'
# }

In [5]:
# nus_categories = ('background','animal', 'barrier', 'bicycle', 'bus', 'car', 'construction_vehicle', 'lane', 
#                   'motorcycle', 'other_surface', 'pedestrian', 'static', 'trailer','truck')

In [6]:
NAME_MAPPING = {
    #1
    'noise':'background',   # perhaps background can be used as static object as it might contain buildings
    'vehicle.ego':'background',
    #2 static objects
#     'noise':'static',
    'flat.driveable_surface':'static',
    'static.manmade':'static',
    'static.other':'static',
    'static.vegetation':'static',
    'static_object.bicycle_rack':'static',
    'flat.other':'static',
    'flat.sidewalk':'static',
    'flat.terrain':'static',
    #3 dynamic objects
    'movable_object.barrier':'background',
    'movable_object.debris' :'background',
    'movable_object.trafficcone': 'background',
    'movable_object.pushable_pullable': 'background',
    'vehicle.bicycle': 'background',
    'vehicle.bus.bendy': 'background',
    'vehicle.bus.rigid': 'background',
    'vehicle.emergency.ambulance': 'background',
    'vehicle.emergency.police': 'background',
    'vehicle.car': 'background',
    'vehicle.construction': 'background',
    'vehicle.motorcycle': 'background',
    'vehicle.trailer': 'background',
    'vehicle.truck': 'background',
    'human.pedestrian.adult':'background',
    'human.pedestrian.child':'background',
    'human.pedestrian.construction_worker':'background',
    'human.pedestrian.police_officer':'background',
    'human.pedestrian.personal_mobility':'background',
    'human.pedestrian.stroller':'background',
    'human.pedestrian.wheelchair':'background',
    'animal':'background',
}

In [7]:
nus_categories = ('background', 'static')

In [8]:
print('Process category information')
categories = []
categories = [dict(id=nus_categories.index(cat_name), name=cat_name) for cat_name in nus_categories]
cat2id = {k_v['name']: k_v['id'] for k_v in categories}

print('Categories (List of dict):\n', categories)
print('Cat2ID Dict:\n', cat2id)

Process category information
Categories (List of dict):
 [{'id': 0, 'name': 'background'}, {'id': 1, 'name': 'static'}]
Cat2ID Dict:
 {'background': 0, 'static': 1}


In [9]:
# dict_colors = {'background':(0, 0, 0),
#                'animal':(70, 130, 180),
#                'barrier': (112, 128, 144),
#                'bicycle':(220, 20, 60),
#                'bus':(255, 69, 0),
#                'car':(255, 158, 0),
#                'construction_vehicle':(233, 150, 70),
#                'lane':(0, 207, 191),
#                'motorcycle': (255, 61, 99),
# #                    'other_surface': 
#                'pedestrian':(0, 0, 230),               
#                    'static':(222, 184, 135),
#                    'truck':(255, 99, 71),
#                    'trailer':(255, 140, 0)
#                }
# print("Color Keys \n", dict_colors.keys())

In [10]:
# palette = []
# for _,v in dict_colors.items():
#     palette.append(list(v))
# print("Color Palette \n", palette)

In [ ]:
for ver in VERSIONS:
    nuim = NuImages(dataroot=DATAROOT,version=ver,verbose=True,lazy=True)
    name_to_index = name_to_index_mapping(nuim.category)
    print("Original Name To Inder Mapping \n", name_to_index)
    images = []
    print('Process image meta information...')
    for sample_info in nuim.sample_data:
        if sample_info['is_key_frame']:
            img_idx = len(images)
            images.append(
                dict(id=img_idx, token=sample_info['token'], file_name=sample_info['filename'],
                     width=sample_info['width'], height=sample_info['height']))

    for img_info in images:
        sd_token = img_info['token']
        image_id = img_info['id']
        # Get image data.
        width, height = img_info['width'], img_info['height']
        semseg_mask = np.zeros((height, width)).astype('uint8')

        # Load stuff / surface regions.
        surface_anns = [
            o for o in nuim.surface_ann if o['sample_data_token'] == sd_token]
        for ann in surface_anns:
            # Get color and mask.
            category_token = ann['category_token']
            category_name = nuim.get('category', category_token)['name']
            if ann['mask'] is None:
                continue
            mask = mask_decode(ann['mask'])

            # Draw mask for semantic segmentation.
            semseg_mask[mask == 1] = cat2id[NAME_MAPPING[category_name]]
        # Load object instances.
        object_anns = [
            o for o in nuim.object_ann if o['sample_data_token'] == sd_token]
        # Sort by token to ensure that objects always appear in the
        # instance mask in the same order.
        object_anns = sorted(object_anns, key=lambda k: k['token'])
        for i, ann in enumerate(object_anns):
            # Get color, box, mask and name.
            category_token = ann['category_token']
            category_name = nuim.get('category', category_token)['name']
            if ann['mask'] is None:
                continue
            mask = mask_decode(ann['mask'])

            # Draw masks for semantic segmentation and instance segmentation.
            semseg_mask[mask == 1] = cat2id[NAME_MAPPING[category_name]]
#         seg_img = Image.fromarray(semseg_mask).convert('P')
#         seg_img.putpalette(np.array(palette, dtype = np.uint8))
        seg_img = Image.fromarray(semseg_mask, mode = 'L')
        file_name = img_info['file_name']

        img_file_name = os.path.normpath(file_name).split(os.sep)[-1]
        seg_file_name = img_file_name.replace('jpg','png')
        split_file_name = img_file_name.split('.')

        shutil.copy(os.path.join(DATAROOT,file_name), os.path.join(data_root,img_dir,img_file_name))
        seg_img.save(os.path.join(data_root,ann_dir,seg_file_name))
        ver_type = ver.split('-')[-1]
        #print(ver_type)
        with open(osp.join(data_root,split_dir,ver_type+'txt'),'a+') as f:
            f.seek(0)
            d = f.read(100)
            if len(d) > 0:
                f.write("\n")
            f.write(split_file_name[0])            

Loading nuImages tables for version v1.0-train...
Done loading in 0.000 seconds (lazy=True).
Loaded 25 category(s) in 0.000s,
Original Name To Inder Mapping 
 {'animal': 1, 'human.pedestrian.adult': 2, 'human.pedestrian.child': 3, 'human.pedestrian.construction_worker': 4, 'human.pedestrian.personal_mobility': 5, 'human.pedestrian.police_officer': 6, 'human.pedestrian.stroller': 7, 'human.pedestrian.wheelchair': 8, 'movable_object.barrier': 9, 'movable_object.debris': 10, 'movable_object.pushable_pullable': 11, 'movable_object.trafficcone': 12, 'static_object.bicycle_rack': 13, 'vehicle.bicycle': 14, 'vehicle.bus.bendy': 15, 'vehicle.bus.rigid': 16, 'vehicle.car': 17, 'vehicle.construction': 18, 'vehicle.emergency.ambulance': 19, 'vehicle.emergency.police': 20, 'vehicle.motorcycle': 21, 'vehicle.trailer': 22, 'vehicle.truck': 23, 'flat.driveable_surface': 24, 'vehicle.ego': 31}
Process image meta information...
Loaded 872181 sample_data(s) in 2.627s,
Loaded 73755 surface_ann(s) in 1.13